# Optimization Example: A Portfolio Problem
**Author:    David Childers**

**Date:      3-29-2023**

In this document I present several approaches to numerical optimization through the context of a portfolio choice problem with CARA utility. The approach will use in particular the Julia toolbox Optim.jl, along with a few related tools, for which this is intended to serve as an introduction. The problem is taken from Judd, *Computational Methods for Economics Ch. 4* and the code is based on my previous Matlab code for the same problem. Another useful introduction to these tools, to which this example is related, is available from QuantEcon at <https://julia.quantecon.org/more_julia/optimization_solver_packages.html>.

In [1]:
# #Set up the Julia environment to use standard packages using Project and Manifest in document's directory
import Pkg; 
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Library/CloudStorage/Dropbox/Computational Methods/Code/Julia`


In [2]:
#Check status of packages
Pkg.status()

Status `~/Library/CloudStorage/Dropbox/Computational Methods/Code/Julia/Project.toml`
  [28f2ccd6] ApproxFun v0.13.17
  [a134a8b2] BlackBoxOptim v0.6.2
  [e2554f3b] Clp v1.0.2
  [0c46a032] DifferentialEquations v7.7.0
  [31c24e10] Distributions v0.25.87
  [442a2c76] FastGaussQuadrature v0.5.1
  [f6369f11] ForwardDiff v0.10.35
  [7073ff75] IJulia v1.24.0
  [a98d9a8b] Interpolations v0.14.7
  [b6b21f68] Ipopt v1.2.1
  [4076af6c] JuMP v1.10.0
  [0fc2ff8b] LeastSquaresOptim v0.8.4
  [d41bc354] NLSolversBase v7.8.3
  [2774e3e8] NLsolve v4.5.1
  [429524aa] Optim v1.7.5
  [d96e819e] Parameters v0.12.3
  [91a5bcdd] Plots v1.38.9
  [8a4e6c94] QuasiMonteCarlo v0.2.19
  [f2b01f46] Roots v2.0.10
  [e88e6eb3] Zygote v0.6.60
  [37e2e46d] LinearAlgebra
  [de0858da] Printf
  [9a3f8284] Random
  [10745b16] Statistics


In [3]:
#Uncomment and execute following line if any of the following packages are missing
#Pkg.add(LinearAlgebra, Statistics, ForwardDiff, Zygote, Optim, JuMP, Ipopt, BlackBoxOptim, Roots, NLsolve, LeastSquaresOptim)
#Pkg.status()

In [4]:
#Load packages to be used for Optimization
using LinearAlgebra, Statistics #Basic math
#Specialized differentiation, optimization, and equation solving libraries
using ForwardDiff, Zygote, Optim, JuMP, Ipopt, BlackBoxOptim, Roots, NLsolve, LeastSquaresOptim
using Optim: converged, maximum, maximizer, minimizer, iterations #some functions from the Optim library

For this example, we first declare the function we want to optimize. The function represents a one-period portfolio optimization problem over a vector of $n$ assets, which each have a current price `assetprice` and pay out a certain defined payoff in each of $S$ total states, stored as $n\times S$ array `assetreturns`. The investor is endowed with a vector of `endowments` of each asset, and receives payoff given by the utility of initial period consumption plus expected utility of returns, where expectation is taken with respect to a probability distribution over states with probability mass vector `stateprobs` and utility of returns is CARA with coefficient of absolute risk aversion `cara`. The choice the investor faces is to find the optimal vector of portfolio allocations to each asset, `pweights` by maximizing this utility(respectively, minimizing $-1$ times this utility.

In [5]:
#Our minus expected utility function, that the investor would like to minimize

function portfolio_return(pweights; stateprobs, endowment, assetprice, cara = 2)
    stateutil = -exp.(-cara*(pweights'*assetreturns))
    return mutility = exp(cara*assetprice'*(pweights-endowment))-sum(stateutil*stateprobs)
end

portfolio_return (generic function with 1 method)

In [6]:
# Try some example parameter values to test the function out

cara = 2 #This was set as a default, so you don't have to pass it to the function, but you can if you want
stateprobs = [0.1, 0.2, 0.7] #Probability of each state

endowment=[0.1, 0.1]    #Endowment of each asset
assetprice=[1, 1]                #Price of each asset
assetreturns=[2 1 0.1;         #Returns of each asset in each state
             0 1 2] 

guessweight=[0.1,0.1]; #Initial guess of possible portfolio weights

In [7]:
# Evaluate directly
stateutil = -exp.(-cara*(guessweight'*assetreturns))
guessutility = exp(cara*assetprice'*(guessweight-endowment))-sum(stateutil*stateprobs)

1.6610287876812315

In [8]:
mutilityguess = portfolio_return(guessweight, stateprobs=stateprobs, endowment=endowment, assetprice=assetprice, cara = 2)

1.6610287876812315

In [9]:
# Declare an anonymous function, which takes only the portfolio choice as input
portreturn = x->portfolio_return(x,stateprobs=stateprobs, endowment=endowment, assetprice=assetprice, cara = 2)
# Use default optimization, with nothing more than an initial guess
result = optimize(portreturn, guessweight)

 * Status: success

 * Candidate solution
    Final objective value:     1.410094e+00

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    28
    f(x) calls:    59


With no gradient, `optimize` runs the Nelder-Mead "Amoeba" method, which takes quite a few function calls to reach an optimum.  Unless you know your function is not differentiable, or derivatives are hard to compute, it may be strongly preferable to use a first or even second order method.  To try this, we need a Jacobian, and, for second order, a Hessian. In this problem, these have analytical forms, so I will provide them. I will then compare to results produced by automatic differentiation.

In [10]:
#Declare Gradient function: use an in-place, or "non-allocating" function
# ! means it takes a preallocated vector and changes it, rather than creating a new vector
#This saves memory, and can also improve speed due to reducing time in allocating new vectors
# Note here we are defining this function without the additional arguments: they are taken from the memory environment
# We would have to define again, or use an anonymous function, to change the constant choices
function portGrad!(J,pweights)
    stategrad = repeat(cara*exp.(-cara*(pweights'*assetreturns)),length(assetprice)).*assetreturns
    return J .= cara*exp.(cara*assetprice'*(pweights-endowment))*assetprice-stategrad*stateprobs
end

portGrad! (generic function with 1 method)

In [11]:
# Evaluate at a particular parameter choice
GradG = copy(guessweight)
portGrad!(GradG,guessweight)

2-element Vector{Float64}:
  1.3717574083973805
 -0.10785911389641445

In [12]:
#Call optimizer with gradient function
result1 = optimize(portreturn,portGrad!,guessweight)

 * Status: success

 * Candidate solution
    Final objective value:     1.410094e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 4.62e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.80e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.37e-13 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    6
    f(x) calls:    17
    ∇f(x) calls:   17


With one derivative given, by default `Optim.jl` calls a first order method, here L-BFGS, a version of BFGS which does not store the entire Broyden approximation of the Jacobian. This is probably the most commonly used first order quasi-Newton method, with good tradeoff between number of iterations and iteration complexity. We see it achieved the same minimum in many fewer iterations and function calls than Nelder Mead, though it did require gradient calls.

Analytic gradients can sometimes be a pain to code up, so it would be useful to also apply automatic differentiation.

In [13]:
# Compare to result from ForwardDiff, which computes Gradient by forward mode automatic differentiation

GG = ForwardDiff.gradient(portreturn,guessweight)

2-element Vector{Float64}:
  1.3717574083973805
 -0.10785911389641445

In [14]:
# To pass to optimizer, call as an anonymous function again, and also make non-allocating
# ! means it takes a preallocated vector and changes it, rather than creating a new vector
function pgrad!(G,x)
    return G.=ForwardDiff.gradient(portreturn,x)
end

pgrad! (generic function with 1 method)

In [15]:
#Evaluate Gradient
pgrad!(GradG,guessweight) #Should be same as analytical formula

2-element Vector{Float64}:
  1.3717574083973805
 -0.10785911389641445

In [16]:
#Call optimizer with Forward Diff gradient function
result2 = optimize(portreturn,pgrad!,guessweight)

 * Status: success

 * Candidate solution
    Final objective value:     1.410094e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 4.62e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.80e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.37e-13 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    6
    f(x) calls:    17
    ∇f(x) calls:   17


That produced exactly the same result, at more or less similar speed, with no coding. You can call `optimize` without supplying a gradient function, but still choose a first order method like L-BFGS, and it will use a finite difference procedure to compute gradients. If the function is amenable to automatic differentiation, you can also call ForwardDiff as an option.

In [17]:
#Call optimizer with finite difference gradient function, by choosing option LBFGS()
result3 = optimize(portreturn,guessweight,LBFGS())

 * Status: success

 * Candidate solution
    Final objective value:     1.410094e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 4.62e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.80e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.22e-16 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.57e-16 ≰ 0.0e+00
    |g(x)|                 = 0.00e+00 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    6
    f(x) calls:    18
    ∇f(x) calls:   18


In [18]:
result4 = optimize(portreturn,guessweight,LBFGS(),autodiff=:forward)

 * Status: success

 * Candidate solution
    Final objective value:     1.410094e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 4.62e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.80e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.37e-13 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    6
    f(x) calls:    17
    ∇f(x) calls:   17


Autodiff saved us one function call.

In [19]:
# We can use first order methods other than L-BFGS. Try, e.g., ConjugateGradient()
result5 = optimize(portreturn,guessweight,ConjugateGradient(),autodiff=:forward)

 * Status: success

 * Candidate solution
    Final objective value:     1.410094e+00

 * Found with
    Algorithm:     Conjugate Gradient

 * Convergence measures
    |x - x'|               = 1.96e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 7.60e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.22e-16 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.57e-16 ≰ 0.0e+00
    |g(x)|                 = 3.33e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    7
    f(x) calls:    16
    ∇f(x) calls:   9


Different first order methods may have differing degrees of speed and reliability on different problems (e.g., conjugate gradient works well with sparse or low rank problems), but require similar user inputs. Second order methods, like Newton's method, may gain speed at the cost of requiring a Hessian. This can be user provided or automatic as before.

In [20]:
M=assetreturns[:,1]*assetreturns[:,1]' #A matrix the same size as the Hessian just as a placeholder

2×2 Matrix{Float64}:
 4.0  0.0
 0.0  0.0

In [21]:
#Calculate Analytical Hessian formula
function portHess!(H,pweights)
    #Calculate Hessian of expected utility by adding over states
    nextperHess=zeros(size(H))
    for i in 1:length(stateprobs)
        nextperHess += stateprobs[i]*cara^2*exp(-cara*(pweights'*assetreturns[:,i]))*(assetreturns[:,i]*assetreturns[:,i]')
    end
    #Calculate Hessian of current period utility
    hess1= cara^2*exp(cara*assetprice'*(pweights-endowment))*(assetprice*assetprice')
    #Hessian of total utility of choice
    return H .= nextperHess + hess1
end

portHess! (generic function with 1 method)

In [22]:
portHess!(M,guessweight)

2×2 Matrix{Float64}:
 5.62717   4.9042
 4.9042   11.8952

In [23]:
#Use this in optimize to apply a second order method
result6 = optimize(portreturn,portGrad!,portHess!,guessweight)

 * Status: success

 * Candidate solution
    Final objective value:     1.410094e+00

 * Found with
    Algorithm:     Newton's Method

 * Convergence measures
    |x - x'|               = 2.57e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.97e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.66e-15 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.89e-15 ≰ 0.0e+00
    |g(x)|                 = 5.11e-15 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    4
    f(x) calls:    12
    ∇f(x) calls:   12
    ∇²f(x) calls:  4


Newton's method converged in only 4 iterations, with 12 function and 12 gradent calls, but also required a Hessian. It was also notably slower than a quasi-Newton method, even with user-provided Hessian, which is a common occurence with Newton's method, since it requires Hessian evaluations and linear system solves each iteration.

In [24]:
#As with gradients, we can get Hessians from automatic differentiation
ForwardDiff.hessian(portreturn,guessweight)

2×2 Matrix{Float64}:
 5.62717   4.9042
 4.9042   11.8952

In [25]:
#We can use these in our optimizer by turning into an anonymous, in-place function
function phess!(H,x)
    return H.=ForwardDiff.hessian(portreturn,x)
end

#Call Newton's method
result7 = optimize(portreturn,pgrad!,phess!,guessweight)

 * Status: success

 * Candidate solution
    Final objective value:     1.410094e+00

 * Found with
    Algorithm:     Newton's Method

 * Convergence measures
    |x - x'|               = 2.57e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.97e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.66e-15 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.89e-15 ≰ 0.0e+00
    |g(x)|                 = 5.11e-15 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    4
    f(x) calls:    12
    ∇f(x) calls:   12
    ∇²f(x) calls:  4


## Structured Optimization and JuMP
For generic optimization problems, algorithms which make few assumptions beyond the presence of local information can be reasonable. However, many problems have global structure which permits specialized optimization methods, which can perform better than generic methods, often overwhelmingly so. Problems with linear, quadratic, conic, semidefinite matrix, etc objectives, and linear, quadratic, integer, etc, constraints, have specialized methods available, often implemented in specialized software, including high quality commercial libraries like *Knitro* and *Gurobi*. For these programs, it is helpful to have a specialized modeling language which can express a problem in a suitable format for such a solver. **AMPL** is a commercial program which provides such a language, with interface to many solvers, available to try at <https://ampl.com/try-ampl/try-ampl-online/>. In Julia, the **JuMP** library has similar function, providing a modeling language and interface with external solvers.

To see how it looks like, I will translate a linear programming example from the AMPL library: for more details on JuMP, see [the documentation](https://www.juliaopt.org/JuMP.jl/stable/).

In [26]:
# Set problem parameters

avail = 40 #Total hours available
#For each product, set tons produced per hour, profit per ton, and limit on tons sold in a week
param = Dict("bands"=>( rate = 200, profit = 25, market = 6000 ), 
             "coils"=>( rate = 140, profit = 30, market = 4000 ) )
PROD = collect(keys(param)) #Products
pindex = 1:length(PROD) #Set of products

1:2

In [27]:
#Build model in JuMP, with interior point optimizer Ipopt
steel=Model(Ipopt.Optimizer) # with_optimizer deprecated

#Declare variable to optimize, make, which is nonnegative 
@variable(steel,make[pindex]>=0,container=Array) 

2-element Vector{VariableRef}:
 make[1]
 make[2]

In [28]:
#Add market constraints
@constraint(steel,marketconstraint[p=pindex],make[p]<=param[PROD[p]][:market])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:2
And data, a 2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 marketconstraint[1] : make[1] ≤ 4000.0
 marketconstraint[2] : make[2] ≤ 6000.0

In [29]:
#Add time constraint
@constraint(steel,timeconstraint,sum(make[p]/param[PROD[p]][:rate] for p in pindex)<=avail)

timeconstraint : 0.007142857142857143 make[1] + 0.005 make[2] ≤ 40.0

In [30]:
#Set objective function: maximize profit
@objective(steel,Max,sum(make[p]*param[PROD[p]][:profit] for p in pindex))

30 make[1] + 25 make[2]

In [31]:
#optimize using declared solver
JuMP.optimize!(steel)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

In [32]:
#Optimized objective: total profit
objective_value(steel)

192000.00191999497

In [33]:
#Optimal values of decisions
for p in pindex
    println("Use $(value(make[p])) units of $(PROD[p]) in optimal production plan.")
end

Use 1400.0000140003542 units of coils in optimal production plan.
Use 6000.000059999374 units of bands in optimal production plan.


In [34]:
#If Clp optimizer is missing, uncomment the following to install it.
#Pkg.add(Clp)

In [35]:
using Clp

In [36]:
#Set optimizer as Clp, with Primal Simplex algorithm as solver: see https://github.com/JuliaOpt/Clp.jl
steel2 = Model(optimizer_with_attributes(Clp.Optimizer, "Algorithm" => 1))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Clp

In [37]:
#Declare model again
@variable(steel2,make[pindex]>=0,container=Array) 
@constraint(steel2,marketconstraint[p=pindex],make[p]<=param[PROD[p]][:market])
@constraint(steel2,timeconstraint,sum(make[p]/param[PROD[p]][:rate] for p in pindex)<=avail)
@objective(steel2,Max,sum(make[p]*param[PROD[p]][:profit] for p in pindex))
steel2

A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Clp
Names registered in the model: make, marketconstraint, timeconstraint

In [38]:
#optimize using declared solver
@show JuMP.optimize!(steel2)

JuMP.optimize!(steel2) = nothing
Coin0506I Presolve 1 (-2) rows, 2 (0) columns and 2 (-2) elements
Clp0006I 0  Obj -0 Dual inf 65.714284 (2)
Clp0006I 1  Obj 192000
Clp0000I Optimal - objective value 192000
Coin0511I After Postsolve, objective 192000, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 192000 - 1 iterations time 0.002, Presolve 0.00


In [39]:
objective_value(steel2)

192000.0

In [40]:
#Optimal values of decisions
for p in pindex
    println("Use $(value(make[p])) units of $(PROD[p]) in optimal production plan.")
end

Use 1400.000000000001 units of coils in optimal production plan.
Use 5999.999999999999 units of bands in optimal production plan.


In [41]:
#Are these integers, to machine precision?
value(make[1])≈1400

true

In [42]:
value(make[2])≈6000

true

Switching from an interior point solver, as in Ipopt, to a simplex method solver resulted in exact solutions, up to machine precision. The reason is that the simplex method moves along constraints, while an interior point solver, as the name suggests, gradually relaxes a penalty that keeps it always strictly inside the constraints. A simplex method is fast and exact "most of the time" in a formal sense, but can behave poorly on worst case problems. Interior point methods for linear programs instead give good approximation guarantees in all cases.